In [2]:
import os
import re
import pandas as pd
import argparse
import glob
import json
import datetime
import subprocess

In [3]:
fof_file = "./all_fastq.fof"
# get fof file
with open(fof_file, 'r') as fi:
    fof = fi.read().splitlines()    

In [4]:
manifest = "./manifest_20180227.txt"
# Read the manifest into a dataframe
df = pd.read_csv(manifest, header=0, delimiter="\t")
# Add a column with the full name of the fastq file
df["fullname"] = df["path"].str.split('/').str[-1]
df.head()

,path,batch,read,sample_id,seq_id,seq_id_read,sample_id_ref,kit,flowcell,lane,sample_id_ref_rel,fullname
0,/nfs/production2c/asj/projects/ATA_CRG/raw_dat...,crg1,1,10ss,C70UYACXX_8_10ss,C70UYACXX_8_10ss_1,10ss,sureselect,C70UYACXX,8,ATA_003,C70UYACXX_8_10ss_1.fastq.gz
1,/nfs/production2c/asj/projects/ATA_CRG/raw_dat...,crg1,2,10ss,C70UYACXX_8_10ss,C70UYACXX_8_10ss_2,10ss,sureselect,C70UYACXX,8,ATA_003,C70UYACXX_8_10ss_2.fastq.gz
2,/nfs/production2c/asj/projects/ATA_CRG/raw_dat...,crg1,1,11ss,C70UYACXX_8_11ss,C70UYACXX_8_11ss_1,11ss,sureselect,C70UYACXX,8,ATA_006,C70UYACXX_8_11ss_1.fastq.gz
3,/nfs/production2c/asj/projects/ATA_CRG/raw_dat...,crg1,2,11ss,C70UYACXX_8_11ss,C70UYACXX_8_11ss_2,11ss,sureselect,C70UYACXX,8,ATA_006,C70UYACXX_8_11ss_2.fastq.gz
4,/nfs/production2c/asj/projects/ATA_CRG/raw_dat...,crg1,1,13ss,C70UYACXX_8_13ss,C70UYACXX_8_13ss_1,13ss,sureselect,C70UYACXX,8,ATA_020,C70UYACXX_8_13ss_1.fastq.gz


In [5]:
# sort the dataframe so we have them grouped by sample and in order for concatenating.
df_fastq_sorted = df.sort_values(by=["sample_id_ref_rel", "read", "lane", "batch"]).reset_index(drop=True)
df_fastq_sorted

,path,batch,read,sample_id,seq_id,seq_id_read,sample_id_ref,kit,flowcell,lane,sample_id_ref_rel,fullname
0,/nfs/production3c/scratch_illu_runs/141201_H11...,incliva1,1,1,ATA_1_TAAGGCG_L001,ATA_1_TAAGGCG_L001_1,ATA_001,nextera,C26YAACXX,1,ATA_001,ATA_1_TAAGGCG_L001_R1_001.fastq.gz
1,/nfs/production3c/scratch_illu_runs/141112-hea...,hic1,1,1,ATA_1_TAAGGCG-GTAAGGA_L002,ATA_1_TAAGGCG-GTAAGGA_L002_1,ATA_001,nextera,HAYDAADXX,2,ATA_001,ATA_1_TAAGGCG-GTAAGGA_L002_R1_001.fastq.gz
2,/nfs/production3c/scratch_illu_runs/141126-hea...,hic2,1,1,ATA_1_TAAGGCGA_L002_run2,ATA_1_TAAGGCGA_L002_run2_1,ATA_001,nextera,HB00FADXX,2,ATA_001,ATA_1_TAAGGCGA_L002_R1_001.fastq.gz
3,/nfs/production3c/scratch_illu_runs/141210-hea...,hic4,1,1,ATA_1_TAAGGCGA_L002_run4,ATA_1_TAAGGCGA_L002_run4_1,ATA_001,nextera,HB1T8ADXX,2,ATA_001,ATA_1_TAAGGCGA_L002_R1_001.fastq.gz
4,/nfs/production3c/scratch_illu_runs/141201_H11...,incliva1,1,1,ATA_1_TAAGGCG-GTAAGGA_L003,ATA_1_TAAGGCG-GTAAGGA_L003_1,ATA_001,nextera,C26YAACXX,3,ATA_001,ATA_1_TAAGGCG-GTAAGGA_L003_R1_001.fastq.gz
5,/nfs/production3c/scratch_illu_runs/141201_H11...,incliva1,1,1,ATA_1_TAAGGCG_L004,ATA_1_TAAGGCG_L004_1,ATA_001,nextera,C26YAACXX,4,ATA_001,ATA_1_TAAGGCG_L004_R1_001.fastq.gz
6,/nfs/production3c/scratch_illu_runs/141201_H11...,incliva1,1,1,ATA_1_TAAGGCG-GTAAGGA_L006,ATA_1_TAAGGCG-GTAAGGA_L006_1,ATA_001,nextera,C26YAACXX,6,ATA_001,ATA_1_TAAGGCG-GTAAGGA_L006_R1_001.fastq.gz
7,/nfs/production3c/scratch_illu_runs/141201_H11...,incliva1,1,1,ATA_1_TAAGGCG-GTAAGGA_L007,ATA_1_TAAGGCG-GTAAGGA_L007_1,ATA_001,nextera,C26YAACXX,7,ATA_001,ATA_1_TAAGGCG-GTAAGGA_L007_R1_001.fastq.gz
8,/nfs/production2c/asj/projects/ATA_CRG/raw_dat...,crg1,1,9ss,C70UYACXX_8_9ss,C70UYACXX_8_9ss_1,9ss,sureselect,C70UYACXX,8,ATA_001,C70UYACXX_8_9ss_1.fastq.gz
9,/nfs/production2c/asj/projects/ATA_CRG/raw_dat...,crg1,1,9ss,C76KMACXX_8_9ss,C76KMACXX_8_9ss_1,9ss,sureselect,C76KMACXX,8,ATA_001,C76KMACXX_8_9ss_1.fastq.gz


In [10]:
# Create the zcat commands
cmd_time = datetime.datetime.now().strftime("%Y%m%d_%H-%M-%S")
cmd_sh = "cmd_zcat_fastqATA_{}.sh".format(cmd_time)

# Create list by sample
samples = list(df_fastq_sorted['sample_id_ref_rel'].unique())

# Set zcat template
for i in range(len(samples)):
    sample_fastqs = df_fastq_sorted.loc[df_fastq_sorted["sample_id_ref_rel"]==samples[i]]
    # Get samplenames, we create a list with the full names and then split it in half (R1 and R2)
    samplenames = list(sample_fastqs['fullname'])
    new_list_samplenames = []
    for line in samplenames:
        nl = '/nfs/ngsbackup2/ata/data/fastq/' + line
        new_list_samplenames.append(nl)
    
    list_R1 = new_list_samplenames[:len(new_list_samplenames)//2]
    list_R2 = new_list_samplenames[len(new_list_samplenames)//2:]
    
    R1_str = "zcat {} | gzip > /nfs/production4b/cmc_projects4b_tmp/{}_R1_merged.fastq.gz".format(' '.join(list_R1), samples[i])
    R2_str = "zcat {} | gzip > /nfs/production4b/cmc_projects4b_tmp/{}_R2_merged.fastq.gz".format(" ".join(list_R2), samples[i])
    
    # Write the cmd.sh file
    with open(cmd_sh, 'a') as cmd_file:
        cmd_file.write(R1_str + '\n')
        cmd_file.write(R2_str + '\n')
        
print("Run this cmd_file in the directory where the fastq files are stored: {}".format(cmd_sh))
    

Run this cmd_file in the directory where the fastq files are stored: cmd_zcat_fastqATA_20190412_11-03-27.sh
